In [26]:
import pandas as pd
from maskdilation import direct_dilation, extracted_features
from run_radiomics import merge_label_feature
from select_test_feature import merge_all
from RJSNRadiomicsFeatureExtractor import main_run
import os
from tqdm import tqdm
from select_test_feature import external
tasks = ["LNM", "LVSI"]
dilation = {"LVSI":{"DWI": 5, "T1CE": 6, "T2": 3}, "LNM" :{"DWI": 9, "T1CE": 9, "T2": 7}}
modals = ["DWI", "T1CE", "T2"]
splits = ["intra", "merge", "peri"]

In [3]:
# mask dilation
spacing_path = "/homes/syli/dataset/EC_all/outside/new_1.5T/clear_up_spacing"
direct_dilation(spacing_path)

100%|██████████| 50/50 [00:10<00:00,  4.90it/s]


In [ ]:
# extra features
df_path = "/homes/syli/dataset/EC_all/outside/new_1.5T/dataframe"
for modal in modals:
    save_path = os.path.join(df_path, modal)
    os.makedirs(save_path, exist_ok=True)
    for i in tqdm([3, 5, 6,7,9]):
        save = os.path.join(save_path, f"{modal}_{i}_features.csv")
        main_run("/homes/syli/dataset/EC_all/outside/new_1.5T/clear_up_spacing", f"{modal}_trans", f"{modal}_dilation_{i}.nii.gz", save)


In [14]:
# 先找到对应文件，进行zscore, 再merge,
root = "/homes/syli/dataset/EC_all/outside"
clinical_path = root+"/1.5 new needed.xlsx"
def zscore(data, zscore_df):
    data.iloc[:, 1:] = (data.iloc[:, 1:] - zscore_df.loc["mean"]) / zscore_df.loc["std"]
    return data
for task in tasks:
    for modal in modals:
        os.makedirs(root+f"/{task}/{modal}/intra", exist_ok=True)
        os.makedirs(root+f"/{task}/{modal}/peri", exist_ok=True)
        os.makedirs(root+f"/{task}/{modal}/merge", exist_ok=True)
        zscore_df = pd.read_csv(f"/homes/syli/dataset/EC_all/{task}_model/liuzhou_split/{modal}/dilation_{dilation[task][modal]}/mean_std.csv", index_col=0)
        data = pd.read_csv(df_path+f"/{modal}/{modal}_{dilation[task][modal]}_features.csv")
        features = [i.replace("trans", f"resampled.nii") for i in list(data)]
        data.columns = features
        liuzhou_df = zscore(data, zscore_df)
        liuzhou_df.to_csv(root+f"/{task}/{modal}/peri/zscore_features.csv")

        zscore_df = pd.read_csv(f"/homes/syli/dataset/EC_all/{task}_model/liunei/{modal}/mean_std.csv", index_col=0)
        data = pd.read_csv(df_path+f"/{modal}_features.csv")
        features = [i.replace("trans", f"resampled.nii") for i in list(data)]
        data.columns = features
        liunei_df = zscore(data, zscore_df)
        liunei_df.to_csv(root+f"/{task}/{modal}/intra/zscore_features.csv")

        features = [i.replace("resampled.nii", f"dilation_{dilation[task][modal]}") for i in list(liuzhou_df)]
        liuzhou_df.columns = features
        liunei_df["CaseName"] = liunei_df["CaseName"].astype(dtype="str")
        liuzhou_df["CaseName"] = liuzhou_df["CaseName"].astype(dtype="str")
        sum_df = pd.merge(liunei_df, liuzhou_df, on=["CaseName"], validate="one_to_one")
        modal_store = root+f"/{task}/{modal}/merge"
        sum_df.to_csv(os.path.join(modal_store, "zscore_features.csv"), index=False)


In [41]:
clinical_df = pd.read_excel(clinical_path)
clinical_df["CaseName"] = clinical_df["CaseName"].astype(dtype="str")
clinical_df["CaseName"] = [i.strip() for i in clinical_df["CaseName"].values]
print(len(clinical_df))
for key in tasks:
    for modal in modals:
        for split in ["intra", "peri", "merge"]:
            store_path = os.path.join(root, key, modal, split)
            df = pd.read_csv(os.path.join(store_path, f"zscore_features.csv"))
            df["CaseName"] = df["CaseName"].astype(dtype="str")
            df["CaseName"] = [i.strip() for i in df["CaseName"].values]
            new_df = pd.merge(clinical_df[["CaseName", key]], df)
            print(len(new_df))
            new_df.rename(columns={key: "label"}, inplace=True)
            new_df.to_csv(os.path.join(store_path, f"external_test.csv"), index=False)

54
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49


In [44]:
for task in tasks:
    for modal in modals:
        for split in splits:
            external(os.path.join(root, task, modal, split, "external_test.csv"), 
                    os.path.join("/homes/syli/dataset/EC_all/result", task, modal, split))

In [45]:
for task in tasks:
    for i in splits:    
        flag = True
        for modal in modals:
            pred_df = pd.read_csv(os.path.join(root, task, modal, i, "external_prediction.csv"))
            pred_df.rename(columns={"Pred": f"{modal}_prediction"}, inplace=True)
            if flag:
                new_df = pred_df
                flag = False
            else:
                new_df = pd.merge(new_df, pred_df, on=["CaseName", "label"])
        os.makedirs(os.path.join(root, task, "combine", i), exist_ok=True)
        new_df.to_csv(os.path.join(root, task, "combine", i, "external_features.csv"), index=False)

In [46]:
for task in tasks:
    print(task)
    for split in splits:
        print(split)
        external(os.path.join(root, task, "combine", split, "external_features.csv"), 
                os.path.join("/homes/syli/dataset/EC_all/result", task, "combine", split))

LNM
intra
merge
peri
LVSI
intra
merge
peri


In [62]:
from atuo_radiomics.Metric import EstimatePrediction
for task in tasks:
    print(task)
    for split in splits:
        df = pd.read_csv(os.path.join(root, task, "combine", split, "external_prediction.csv"))
        train_df = pd.read_csv(os.path.join("/homes/syli/dataset/EC_all/result", task, "combine", split, "train_prediction.csv"))
        #new_df =pd.concat((df.loc[:10],df.loc[18:]))
        new_df = df.loc[:10]
        #print(new_df.head())
        train_metrics = EstimatePrediction(train_df["Pred"].values, train_df["label"].values.astype(int))
        metrics = EstimatePrediction(new_df["Pred"].values, new_df["label"].values.astype(int), "", eval(train_metrics["Cutoff"]))
        print(split, "AUC:", metrics["AUC"])


LNM
intra AUC: 0.7222
merge AUC: 0.7222
peri AUC: 0.6667
LVSI
intra AUC: 0.6250
merge AUC: 0.6250
peri AUC: 0.5833
